In [10]:
import logging
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBClassifier

from heamy.dataset import Dataset
from heamy.estimator import Regressor, Classifier
from heamy.pipeline import ModelsPipeline

np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)

np.random.seed(1000)
# logging.basicConfig(level=logging.DEBUG)

In [7]:

# load boston dataset from sklearn
data = load_boston()
# print(data)
X, y = data['data'], data['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=111)
# create dataset
dataset = Dataset(X_train,y_train,X_test)

# initialize RandomForest & LinearRegression 
model_rf = Regressor(dataset=dataset, estimator=RandomForestRegressor, parameters={'n_estimators': 50},name='rf')
model_lr = Regressor(dataset=dataset, estimator=LinearRegression, parameters={'normalize': True},name='lr')

# Stack two models 
# Returns new dataset with out-of-fold predictions
pipeline = ModelsPipeline(model_rf,model_lr)
stack_ds = pipeline.stack(k=10,seed=111)

# Train LinearRegression on stacked data (second stage)
stacker = Regressor(dataset=stack_ds, estimator=LinearRegression)
results = stacker.predict()
results
# Validate results using 10 fold cross-validation
# results = stacker.validate(k=10,scorer=mean_absolute_error)

array([25.643243, 25.470019, 28.443487, 18.739014, 46.749256, 24.096496,
       31.391189, 15.830204, 21.105665, 12.702276, 25.97343 , 46.171563,
       20.837093, 26.604365, 14.239824, 23.848273, 26.293446, 20.769526,
       14.181927, 19.142902, 14.472526, 26.5425  , 20.274246, 31.920924,
       11.815633, 19.939913, 43.08211 , 24.73994 , 21.313406, 10.963798,
       29.066465,  8.713377, 12.178138, 23.835774, 44.949629, 24.627297,
        8.607721, 43.27106 , 22.334295, 13.7788  , 23.273812, 18.728429,
       20.404108, 20.801899, 20.960751, 32.759037, 26.422301, 14.175655,
       41.757481, 16.658848, 19.088601])

In [14]:
from heamy.dataset import Dataset
from heamy.estimator import Regressor, Classifier
from heamy.pipeline import ModelsPipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
import pandas as pd
##事先将特征保存到文件中，比较重要，因为jupyter notebook随时会挂掉(⊙_⊙)
train = pd.read_csv("../dealinput/train2ok.csv")
test = pd.read_csv("../dealinput/test2ok.csv")

# train = pd.read_csv('train_part_76_features.csv')
# test = pd.read_csv('test_part_76_features.csv')
col = [c for c in train if c not in ['id','target']]
X_train = train[col]
y_train = train['target'].values 
X_test = test.iloc[:,1:]
"""
##使用基于距离的回归方法需要归一化
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train[col] = scaler.fit_transform(X_train)
X_test[col] = scaler.fit_transform(X_test)
"""
dataset = Dataset(X_train,y_train,X_test)
model_gbrt = Classifier(dataset=dataset, estimator=GradientBoostingClassifier,
                        parameters={'learning_rate': 0.01,'random_state':3,
                                    'n_estimators':200,'subsample':0.8, 
                                    'max_depth' :20 },
                        name='gbrt')
#GradientBoostingRegressor(learning_rate=0.2, random_state=3, n_estimators=200, subsample=0.8, 
#                      max_depth =10)
#model_knn =  Regressor(dataset=dataset, estimator=KNeighborsRegressor, parameters={'n_jobs': -1,'n_neighbors':3},name='knn')
# 
model_xgbc = Classifier(dataset=dataset, estimator=XGBClassifier, 
                        parameters={'learning_rate': 0.01,'random_state':3,
                                'n_estimators':200 ,'subsample':0.8, 
                                    'objective': 'reg:linear','max_depth' :10},
                        name='xgb')
#model_mlp =  Regressor(dataset=dataset, estimator=MLPRegressor, parameters={'hidden_layer_sizes': 10,'random_state':9},name='mlp')
# initialize RandomForest & LinearRegression
model_rf = Classifier(dataset=dataset, estimator=RandomForestClassifier,
                      parameters={'n_estimators': 50},name='rf')
#model_lr = Regressor(dataset=dataset, estimator=LinearRegression, parameters={'normalize': True},name='lr')


# Stack two models
# Returns new dataset with out-of-fold predictions
pipeline = ModelsPipeline(model_rf,model_xgbc)
stack_ds = pipeline.stack(k=5,seed=9)

# Train LinearRegression on stacked data (second stage)
stacker = Regressor(dataset=stack_ds, estimator=LogisticRegression)
results = stacker.predict_proba()


AttributeError: 'Regressor' object has no attribute 'predict_proba'

In [13]:
results

array([0., 0., 0., ..., 0., 0., 0.])